In [1]:
from Models.ModelConversion import prom_stochastic_pnml_to_pnp
from Pytri.PetriNet import PNP, NoInitialMarkingError
from Pytri.ConditionalProb import *
from LKC.LKC import *

import threading
import pandas as pd
import os
import gc
from time import time
from queue import Queue
from operator import itemgetter
import tracemalloc

In [2]:
current_directory = os.getcwd()
one_folders_up = os.path.abspath(os.path.join(current_directory, '..'))
results_folder = os.path.join(one_folders_up, 'Results')
data_folder = os.path.join(one_folders_up, 'Data')

In [3]:
datasets = ["BPI 2020/DomesticDeclarations", "BPI 2020/RequestForPayment", "Sepsis Cases", 
            "Traffic Fines", "Hospital Billing", "BPI 2020/InternationalDeclarations", "BPI 2020/PrepaidTravelCost",
            "BPI 2012","BPI 2020/PermitLog",  "BPI 2017"]

models = ["IMf20_", "IM_", "Heuristic_", "Flower_", "Alpha+_"] #"Alpha_",
types = ["BillClintonEstimator", "AlignmentEstimator"]

shorthand = {"BillClintonEstimator": "bce", "AlignmentEstimator": "align"}

In [4]:
def low_percentage_events(numbers, percentage):
    return sorted([event.replace(" ", "_") for event in numbers.events if numbers.pr_e([event]) <= percentage])

def lowest_events(numbers, amount):
    event_occur = [(event.replace(" ", "_"), numbers.pr_e([event])) for event in numbers.events]
    least_event_occur = sorted(event_occur, key=itemgetter(1))[:amount]
    return sorted([event[0] for event in least_event_occur])

In [5]:
# Note: if the CC calculations is stoped during the spsolve, it might take a while for the thread to end
def reach_calc(pnp, result_queue, stop_event):
    reach = pnp.create_reachability_graph(stop_event)
    if not stop_event.is_set():
        result_queue.put(reach)

def cc_calc(reachability_graph, sensitive, L, upper_cc, lower_cc, result_queue, stop_event):
    cc = paper_cc_model_all(reachability_graph, L, sensitive, upper_cc, lower_cc, stop_event)
    if not stop_event.is_set():
        result_queue.put(cc)

def calculate_cc(dataset, sensitive, cc_type, reach_time_limit = 5, cc_time_limit = 300):
    results = []
    failed_at_results = []
    stop_events = []
    
    for model in models:
        
        # Do some name shifting for the Alpha+ specificly (which is why it is always done last)
        if (dataset[:8] == "BPI 2020") and (model == "Alpha_+"):
            sensitive = [event + "+" for event in sensitive]
            percentage_sensitive = [event + "+" for event in percentage_sensitive]
            number_sensitive = [event + "+" for event in number_sensitive]
        
        for stochastic_type in types:
            
            print("")
            print(model, stochastic_type)
            
            # Try to load the Stochastic PNML file
            try:
                prom_stochastic_pnml_to_pnp(f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnml", 
                                            f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnp")
                pnp = PNP.read_text(f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnp")
            except NoInitialMarkingError:
                print("This one does not have an inital state")
                failed_at_results.append([model, stochastic_type, 0,"Non-InitialState"])
                continue
            except FileNotFoundError:
                print("This one does not exist")
                failed_at_results.append([model, stochastic_type, 0,"Non-Existant"])
                continue
            except:
                print("Something went wrong during loading")
                failed_at_results.append([model, stochastic_type, 0,"Non-Load"])
                continue
            
            print("Loaded the PNP")
            
            # Create a reachability graph. There is a limit of x seconds for this
            # If it takes longer than x seconds we presume the graph will be to big
            result_queue = Queue()
            stop_events.append(threading.Event())
            code_thread = threading.Thread(target=reach_calc, args=(pnp, result_queue, stop_events[-1]))
            code_thread.start()
            code_thread.join(timeout=reach_time_limit)
            
            if code_thread.is_alive():
                stop_events[-1].set()
                #code_thread.join()
                print("Reachability graph calculations timed out")
                failed_at_results.append([model, stochastic_type, 0, "Reach-Timeout"])
                continue

            # If the reachability graph was indeed loaded
            try:
                reachability_graph = result_queue.get()
            except:
                print("Reachability graph calculations failed")
                failed_at_results.append([model, stochastic_type, 0, "Reach-Failure"])
            
            print("Reachability graph calculations succeeded")
            upper_cc, lower_cc = 0,0
            failure = False
            # For L 1-4
            for L in range(1,5):
                result_queue = Queue()
                stop_events.append(threading.Event())
                code_thread = threading.Thread(target=cc_calc, args=(reachability_graph, sensitive, L, upper_cc, lower_cc, result_queue, stop_events[-1]))
                code_thread.start()
                code_thread.join(timeout=cc_time_limit)
                
                if code_thread.is_alive():
                    stop_events[-1].set()
                    #code_thread.join()
                    print(f"CC calculations timed out on L={L}")
                    failed_at_results.append([model, stochastic_type, L, "CC-Timeout"])
                    failure = True
                    break
                
                # If the CC calculations did indeed finish
                try:
                    upper_cc, lower_cc, C = result_queue.get()
                except TypeError:
                    print(f"CC calculations had no deadlock on failed on L={L}")
                    failed_at_results.append([model, stochastic_type, L, "CC-NoDeadlock"])
                    failure = True
                    break
                except:
                    print(f"CC calculations failed on L={L}")
                    failed_at_results.append([model, stochastic_type, L, "CC-Failure"])
                    failure = True
                    break
                
                print(f"CC calculations succeeded for L={L}")
                result_cc = upper_cc/lower_cc if lower_cc != 0 else 0
                results.append([model, stochastic_type, tuple(sensitive), L, result_cc, upper_cc, lower_cc, C])
            
            if not failure:
                failed_at_results.append([model, stochastic_type, 0, "Succes"])

    results_df = pd.DataFrame(results, columns = ["Model", "Stochastic Type", "Sensitive Activities", "Background Size", "Confidence Privacy", "Upper CC", "Lower CC", "Max C"])
    results_df.to_csv(os.path.join(results_folder, f"{dataset}\CC_Model_{cc_type}_{reach_time_limit}_{cc_time_limit}.csv"), index = False)
    
    failed_df = pd.DataFrame(failed_at_results, columns = ["Model", "Stochastic Type", "Background Size", "Problem At"])
    failed_df.to_csv(os.path.join(results_folder, f"{dataset}\CC_Model_Failed_{cc_type}_{reach_time_limit}_{cc_time_limit}.csv"), index = False)

In [6]:
def run_various_cc(numbers, dataset, sensitive, skip = 0):
    sensitive = sorted(sensitive)
    percentage_sensitive = low_percentage_events(numbers, 0.05)
    number_sensitive = lowest_events(numbers, 4)
    
    if skip < 1:
        print(f"\n\033[1m{dataset} Knowledge\033[0m")
        calculate_cc(dataset, sensitive, "Knowledge")
    
    if skip < 2:
        print(f"\n\033[1m{dataset} Percentage\033[0m")
        calculate_cc(dataset, percentage_sensitive, f"Percentage_0.05")
    
    print(f"\n\033[1m{dataset} Amount\033[0m")
    calculate_cc(dataset, number_sensitive, f"Amount_4")

In [8]:
dataset = "Traffic Fines"
sensitive = ["Send_for_Credit_Collection", "Appeal_to_Judge"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "Sepsis Cases"
sensitive = ["Release_A", "Release_B", "Release_C", "Release_D", "Release_E"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "BPI 2012"
sensitive = ["A_DECLINED", "A_CANCELLED", "A_APPROVED"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "BPI 2017"
sensitive = ["A_Denied", "A_Cancelled", "A_Pending"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "BPI 2020/DomesticDeclarations"
sensitive = ["Payment_Handled"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "BPI 2020/RequestForPayment"
sensitive = ["Payment_Handled"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "BPI 2020/InternationalDeclarations"
sensitive = ["Send_Reminder"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "BPI 2020/PrepaidTravelCost"
sensitive = ["Payment_Handled"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "BPI 2020/PermitLog"
sensitive = ["Payment_Handled", "Send_Reminder"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)

dataset = "Hospital Billing"
sensitive = ["DELETE", "REOPEN", "REJECT", "JOIN-PAT"]
numbers = ListIntegerMap.load_from_file(os.path.join(data_folder, f"{dataset}/Number5.json"))
run_various_cc(numbers, dataset, sensitive)


Traffic Fines Knowledge

IMf20_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

IMf20_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

IM_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations timed out on L=3

IM_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations timed out on L=3

Heuristic_ BillClintonEstimator
This one does not exist

Heuristic_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations had no

Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations timed out on L=3

Flower_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Flower_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Alpha+_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations timed out

Alpha+_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations timed out

Sepsis Cases Amount

IMf20_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations timed out on L=4

C:\ProgramData\Anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:206: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


CC calculations succeeded for L=1
CC calculations timed out on L=2

Heuristic_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations timed out on L=2

Flower_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations timed out on L=4

Flower_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations timed out on L=4

Alpha+_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Alpha+_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded fo


BPI 2020/DomesticDeclarations Knowledge

IMf20_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

IMf20_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

IM_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations timed out

IM_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations timed out

Heuristic_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Heuristic_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC ca

CC calculations succeeded for L=3
CC calculations succeeded for L=4

IM_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations timed out

IM_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations timed out

Heuristic_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Heuristic_ AlignmentEstimator
This one does not exist

Flower_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Flower_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Alpha+_ BillClintonEstimator
Loaded the PN

CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations timed out on L=4

Flower_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations timed out on L=4

Alpha+_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Alpha+_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

BPI 2020/PrepaidTravelCost Percentage

IMf20_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations timed out on L=3

IMf20_ A

CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Flower_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Alpha+_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Alpha+_ AlignmentEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations succeeded for L=3
CC calculations succeeded for L=4

Hospital Billing Percentage

IMf20_ BillClintonEstimator
Loaded the PNP
Reachability graph calculations succeeded
CC calculations succeeded for L=1
CC calculations succeeded for L=2
CC calculations 

# Calculations for the odd changes in flower type

In [9]:
dataset = "BPI 2020/DomesticDeclarations"
stochastic_type = "BillClintonEstimator"
model = "Flower_"

prom_stochastic_pnml_to_pnp(f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnml", 
                                            f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnp")
pnp = PNP.read_text(f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnp")

reachability_graph = pnp.create_reachability_graph()

all_labels = set(arc.transition.label for arc in chain.from_iterable(state.outgoing_arcs for state in reachability_graph.states))

sensitive_events = ["Payment_Handled"]

cc = paper_cc_model_all(reachability_graph, 1, sensitive_events, 0, 0)

In [11]:
L = 1
backgrounds = list(combinations(all_labels-set(sensitive_events)-{'None', 'tau'}, 1))

In [12]:
for background in backgrounds:
    background_DFA = create_multiple_DFA(list(background), all_labels-set(background))
    prb = verify_prob(reachability_graph, deepcopy(background_DFA), True)
    
    if prb > epsilon:
        for sensitive_event in sensitive_events:
            full_DFA = add_sensitive_to_background(sensitive_event, deepcopy(background_DFA))
            prsb = verify_prob(reachability_graph, full_DFA, True)
            print(sensitive_event, background, prb, prsb, prsb/prb)

Payment_Handled ('Declaration_REJECTED_by_PRE_APPROVER',) 0.9885057471169911 0.9885049020401607 0.9999991450966949
Payment_Handled ('Request_Payment',) 0.9999004083163342 0.9998506446997092 0.999950231426839
Payment_Handled ('Declaration_FINAL_APPROVED_by_SUPERVISOR',) 0.9999013027934701 0.9998513146157663 0.9999500068881156
Payment_Handled ('Declaration_SAVED_by_EMPLOYEE',) 0.9926470588140286 0.9926457386252135 0.9999986700320085
Payment_Handled ('Declaration_FOR_APPROVAL_by_ADMINISTRATION',) 0.4999999999936018 0.49999999008398505 0.9999999801807665
Payment_Handled ('Declaration_REJECTED_by_SUPERVISOR',) 0.996598639446258 0.9965958179391335 0.9999971688631584
Payment_Handled ('Declaration_SUBMITTED_by_EMPLOYEE',) 0.9999132847632751 0.9998600805411929 0.999946791163901
Payment_Handled ('Declaration_FOR_APPROVAL_by_SUPERVISOR',) 0.4999999999936018 0.49999999008398505 0.9999999801807665
Payment_Handled ('Declaration_REJECTED_by_EMPLOYEE',) 0.9992679355687982 0.9992560259718021 0.99998808

In [17]:
reachability_graph.states[1].outgoing_arcs

[A) RS)P)n20:1 -Declaration_FINAL_APPROVED_by_SUPERVISOR,0.17950671533364043> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_REJECTED_by_SUPERVISOR,0.005191537616499522> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_SAVED_by_EMPLOYEE,0.0023920053864417592> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_FOR_APPROVAL_by_PRE_APPROVER,1.7718558418087103e-05> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_REJECTED_by_PRE_APPROVER,0.0015237960239554908> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_APPROVED_by_PRE_APPROVER,0.012137212516389667> RS)P)n20:1,
 A) RS)P)n20:1 -Payment_Handled,0.17796520075126687> RS)P)n20:1,
 A) RS)P)n20:1 -Request_Payment,0.17789432651759454> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_REJECTED_by_MISSING,0.0016123888160459266> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_FOR_APPROVAL_by_SUPERVISOR,1.7718558418087103e-05> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_REJECTED_by_BUDGET_OWNER,0.0010453949466671393> RS)P)n20:1,
 A) RS)P)n20:1 -Declaration_SUBMITTED_by_EMPLOYEE,0.2043126971189624> RS)P)n20:1,
 A

In [ ]:
dataset = "Sepsis Cases"
stochastic_type = "BillClintonEstimator"
model = "Flower_"

prom_stochastic_pnml_to_pnp(f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnml", 
                                            f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnp")
pnp = PNP.read_text(f"Stochastic Creation Paper Code/spd_we-master/var/{dataset}/osmodel_{shorthand[stochastic_type]}-{model}data.pnp")

reachability_graph = pnp.create_reachability_graph()

all_labels = set(arc.transition.label for arc in chain.from_iterable(state.outgoing_arcs for state in reachability_graph.states))

sensitive_events = ["Release_A", "Release_B", "Release_C", "Release_D", "Release_E"]

In [ ]:
reachability_graph.states[1].outgoing_arcs

In [ ]:
pra = 0.04410121590535655   # release a
pre = 0.06901084456128818   # ER registration
prl = 6.572461386789351e-05 # leaving

print("Release A:", (pra / (pra + prl)))
print("ER Registration:", (pre / (pre + prl)))
print("Both multiplied:", (pra / (pra + prl))*(pre / (pre + prl)))
print("Release A + ER Registration:", ((pra + pre) - ((pra*prl)/(pre+prl)) - ((pre*prl)/(pra+prl))) / (pra + pre + prl))
print("Release A given ER registration", (((pra + pre) - ((pra*prl)/(pre+prl)) - ((pre*prl)/(pra+prl))) / (pra + pre + prl))/(pre / (pre + prl)))
print("Change Release A given ER registration", ((((pra + pre) - ((pra*prl)/(pre+prl)) - ((pre*prl)/(pra+prl))) / (pra + pre + prl))/(pre / (pre + prl)))-(pra / (pra + prl)))
print("Difference?:", (prl / pre)*((pra + pre) - ((pra*prl)/(pre+prl)) - ((pre*prl)/(pra+prl))) / (pra + pre + prl))

In [ ]:
pra/(pra + pre + prl)

In [ ]:
00111111011111110110000000110111
00111111011111111000011000101110

In [ ]:
0.9981411500689543

In [ ]:
(prl / pre)*((pra + pre) - ((pra*prl)/(pre+prl)) - ((pre*prl)/(pra+prl))) / (pra + pre + prl)

In [ ]:
sensitive_DFA = create_multiple_DFA(["Release_A"], all_labels-set(["Release_A"]))
sensitive_DFA.states

In [ ]:
L = 1
backgrounds = list(combinations(all_labels-set(sensitive_events)-{'None', 'tau'}, 1))
paper_cc_model_all(reachability_graph, L, sensitive_events, 0, 0)

In [ ]:
for sensitive_event in sensitive_events:
    sensitive_DFA = create_multiple_DFA([sensitive_event], all_labels-set([sensitive_event]))
    prs = verify_prob(reachability_graph, sensitive_DFA, True)
    print(sensitive_event, prs)
    print("")

In [ ]:
for event in all_labels:
    DFA = create_multiple_DFA([event], all_labels-set([event]))
    prs = verify_prob(reachability_graph, DFA, True)
    print(event, prs)
    print("")

In [ ]:
for background in backgrounds:
    background_DFA = create_multiple_DFA(list(background), all_labels-set(background))
    prb = verify_prob(reachability_graph, deepcopy(background_DFA), True)
    
    if prb > epsilon:
        for sensitive_event in sensitive_events:
            full_DFA = add_sensitive_to_background(sensitive_event, deepcopy(background_DFA))
            prsb = verify_prob(reachability_graph, full_DFA, True)
            print(sensitive_event, background, prb, prsb, prsb/prb, (prsb/prb)-prb)